# Launch an interview
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for simulating qualitative research interviews. In a series of steps we construct an AI agent representing a qualitative researcher and launch an interview with specified topics and interview subjects.

Please also see examples of the `Conversation` [module](https://github.com/expectedparrot/edsl/tree/main/edsl/conversation) which automates the methods used below to simulate a conversation with multiple agents:

* [Buying a car](https://github.com/expectedparrot/edsl/blob/main/edsl/conversation/car_buying.py)
* [Negotiating a mug](https://github.com/expectedparrot/edsl/blob/main/edsl/conversation/mug_negotiation.py)

[EDSL is an open-source library](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

In [1]:
from edsl import QuestionFreeText, Agent, Scenario, Model

import textwrap
from rich import print

EDSL works with many popular language models ([learn more](https://docs.expectedparrot.com/en/latest/language_models.html) about using models). Here we select one to use in running in our survey:

In [2]:
model = Model("gpt-4o")

Here we create methods for constructing agents representing a researcher and subject, and conducting an interview in the form of a survey that we can observe. (Learn more about [designing agents](https://docs.expectedparrot.com/en/latest/agents.html) and [running surveys](https://docs.expectedparrot.com/en/latest/surveys.html)).

In [3]:
def construct_subject(name, traits={}):
    return Agent(name=name, traits=traits)


def construct_researcher(product, topics):
    background = textwrap.dedent(
        f"""\
    You are an expert qualitative researcher. You are researching this product: {product}. 
    You are interested in conducting interviews to learn people's views on the following topics relating 
    to the product: {topics}."""
    )

    instruction = textwrap.dedent(
        f"""\
    You know to ask questions that are appropriate to the age and experience of an interview subject.
    You know to not ask questions that an interview subject would not be able to answer, 
    e.g., if they are a young child, they probably will not be able to answer many questions about prices. 
    You ask excellent follow-up questions.    
    """
    )
    return Agent(traits={"background": background}, instruction=instruction)


def get_next_question(subject, researcher, dialog_so_far):
    scenario = Scenario(
        {"subject": str(subject.traits), "dialog_so_far": dialog_so_far}
    )
    meta_q = QuestionFreeText(
        question_name="next_question",
        question_text="""
        These are the biographic details of the interview subject: {{ subject }}
        This is your current dialog with the interview subject: {{ dialog_so_far }}
        What question you would ask the interview subject next?
        """,
    )
    question_text = (
        meta_q.by(model)
        .by(researcher)
        .by(scenario)
        .run()
        .select("next_question")
        .first()
    )
    return question_text


def get_response_to_question(question_text, subject, dialog_so_far):
    q_to_subject = QuestionFreeText(
        question_name="question",
        question_text=f"""
        This is your current dialog with the interview subject: {dialog_so_far}. 
        You are now being asked:"""
        + question_text,
    )
    response = q_to_subject.by(model).by(subject).run().select("question").first()
    return response


def ask_question(subject, researcher, dialog_so_far):
    question_text = get_next_question(subject, researcher, dialog_so_far)
    response = get_response_to_question(question_text, subject, dialog_so_far)
    return {"question": question_text, "response": response}


def dialog_to_string(d):
    return "\n".join(
        [f"Question: {d['question']}\nResponse: {d['response']}" for d in d]
    )

In [4]:
subject = construct_subject(name="Alice", traits={"gender": "female", "age": 12})
researcher = construct_researcher(product="cassette tapes", topics="technology, music")
result = ask_question(subject, researcher, "")
print(result)

Job UUID,83aba917-07d7-469c-9b2a-508be714f1ae
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/83aba917-07d7-469c-9b2a-508be714f1ae
Error Report URL,None
Results UUID,fa153d8f-18c5-4ed4-ab18-e6d824329063
Results URL,None


Job UUID,e1049ecd-8ae2-425f-9bee-b7c17c7cc748
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/e1049ecd-8ae2-425f-9bee-b7c17c7cc748
Error Report URL,None
Results UUID,c2575f99-b2d4-4fd4-b7f0-351b9817d886
Results URL,None


{
    'question': 'What kind of music do you like to listen to, and how do you usually listen to it?',
    'response': "I really like pop music because it's so catchy and fun to sing along to! I also enjoy some indie 
songs because they have cool vibes. I usually listen to music on my phone using apps like Spotify or YouTube. 
Sometimes, I like to watch music videos too because they're really entertaining!"
}

In [5]:
dialog_so_far = []
for i in range(5):
    result = ask_question(subject, researcher, dialog_to_string(dialog_so_far))
    print(result)
    dialog_so_far.append(result)
print(dialog_so_far)

Job UUID,157ad18d-5f54-4e7f-82c5-a504c55fd770
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/157ad18d-5f54-4e7f-82c5-a504c55fd770
Error Report URL,None
Results UUID,5d98d7c7-e7cf-4d10-b372-555cd2126549
Results URL,None


Job UUID,b88e77bf-0ebc-4d05-8529-6f68aa9d29cd
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/b88e77bf-0ebc-4d05-8529-6f68aa9d29cd
Error Report URL,None
Results UUID,fbe01b15-2aa7-4f1b-a8ec-c36307e9f827
Results URL,None


{
    'question': 'What kind of music do you like to listen to, and how do you usually listen to it?',
    'response': "I really like pop music because it's so catchy and fun to sing along to! I also enjoy some indie 
songs because they have cool vibes. I usually listen to music on my phone using apps like Spotify or YouTube. 
Sometimes, I like to watch music videos too because they're really entertaining!"
}

Job UUID,28d4da77-5c23-47ae-91a5-b7ba4639f45b
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/28d4da77-5c23-47ae-91a5-b7ba4639f45b
Error Report URL,None
Results UUID,ece0d0ea-8ef3-4c2b-a361-052317e4da28
Results URL,None


Job UUID,26be1536-cf11-41f1-b29d-38df87b4083e
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/26be1536-cf11-41f1-b29d-38df87b4083e
Error Report URL,None
Results UUID,a6007a02-fa41-406d-b2dd-99d88237a63d
Results URL,None


...

{
    'question': 'It sounds like your parents have some fun memories with cassette tapes! Have you ever thought 
about making your own mixtape, either using a cassette or maybe digitally, like creating a playlist? What kind of 
songs would you include, and who would you want to share it with?',
    'response': "I've definitely thought about making my own mixtape, but more like a digital playlist since that's
what I'm used to. I think it would be super fun to pick out all my favorite songs and put them together. I'd 
probably include a mix of pop hits that I love singing along to and some indie tracks that have a cool vibe. Maybe 
I'd throw in a few songs from movies or shows I like too! I'd want to share it with my friends so we could all 
listen together, and maybe I'd even make one for my parents with some of their favorite oldies mixed in. It would 
be really cool to see if they like my taste in music!"
}

Job UUID,4ee0ff9d-9f44-4008-990c-7b12672ba505
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/4ee0ff9d-9f44-4008-990c-7b12672ba505
Error Report URL,None
Results UUID,c5a3dbdb-1ae0-4bd4-af31-4f34eeeac8ff
Results URL,None


Job UUID,a61a1958-e1ac-496b-a890-f9e1ececf883
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/a61a1958-e1ac-496b-a890-f9e1ececf883
Error Report URL,None
Results UUID,6f35784d-27ac-4c83-a242-03d077cbcedb
Results URL,None


{
    'question': "That's a great idea to create digital playlists and share them with your friends and family! Since
you're interested in both pop and indie music, I'm curious to know, how do you usually discover new music? Do you 
rely on recommendations from friends, explore playlists on music apps, or maybe find new songs through movies and 
shows you watch?",
    'response': "I usually discover new music in a few different ways! Sometimes my friends will recommend songs to
me, and I love checking them out because we have similar tastes. I also explore playlists on music apps like 
Spotify. They have these cool curated playlists that introduce me to artists and songs I might not have found on my
own. Plus, when I'm watching movies or shows, I often hear songs that I really like and then I look them up later. 
It's always exciting to find new music that way because it feels like a little hidden gem!"
}

[
    {
        'question': 'What kind of music do you like to listen to, and how do you usually listen to it?',
        'response': "I really like pop music because it's so catchy and fun to sing along to! I also enjoy some 
indie songs because they have cool vibes. I usually listen to music on my phone using apps like Spotify or YouTube.
Sometimes, I like to watch music videos too because they're really entertaining!"
    },
    {
        'question': 'Have you ever heard of cassette tapes or seen one before? If so, what do you think about them,
and would you be interested in trying to listen to music on a cassette tape?',
        'response': "Yeah, I've heard of cassette tapes! My parents have talked about them, and I think they 
actually have some old ones at home. I've seen pictures of them, and they look kinda cool and vintage. I think it 
would be fun to try listening to music on a cassette tape just to see what it's like. It seems so different from 
how we listen to music now, and it might be interesting to experience how people used to enjoy music before all the
digital stuff. Plus, it would be like going on a little time travel adventure!"
    },
    {
        'question': "That's great that you're interested in trying out cassette tapes! Since you mentioned that 
your parents have talked about them, have they ever shared any stories or experiences about using cassette tapes 
when they were younger? And do you think it would be fun to listen to music together with them on a cassette 
player?",
        'response': "Yeah, my parents have shared some stories about cassette tapes! They told me how they used to 
make mixtapes by recording songs from the radio or from other tapes. My mom said she would spend a lot of time 
trying to get the timing just right so she wouldn't cut off the beginning or end of a song. They also mentioned how
they had to rewind the tapes with a pencil sometimes, which sounds kind of funny!"
    },
    {
        'question': 'It sounds like your parents have some fun memories with cassette tapes! Have you ever thought 
about making your own mixtape, either using a cassette or maybe digitally, like creating a playlist? What kind of 
songs would you include, and who would you want to share it with?',
        'response': "I've definitely thought about making my own mixtape, but more like a digital playlist since 
that's what I'm used to. I think it would be super fun to pick out all my favorite songs and put them together. I'd
probably include a mix of pop hits that I love singing along to and some indie tracks that have a cool vibe. Maybe 
I'd throw in a few songs from movies or shows I like too! I'd want to share it with my friends so we could all 
listen together, and maybe I'd even make one for my parents with some of their favorite oldies mixed in. It would 
be really cool to see if they like my taste in music!"
    },
    {
        'question': "That's a great idea to create digital playlists and share them with your friends and family! 
Since you're interested in both pop and indie music, I'm curious to know, how do you usually discover new music? Do
you rely on recommendations from friends, explore playlists on music apps, or maybe find new songs through movies 
and shows you watch?",
        'response': "I usually discover new music in a few different ways! Sometimes my friends will recommend 
songs to me, and I love checking them out because we have similar tastes. I also explore playlists on music apps 
like Spotify. They have these cool curated playlists that introduce me to artists and songs I might not have found 
on my own. Plus, when I'm watching movies or shows, I often hear songs that I really like and then I look them up 
later. It's always exciting to find new music that way because it feels like a little hidden gem!"
    }
]